In [13]:
import pandas as pd
import numpy as np
import cv2
from datetime import datetime
from PIL import Image
from IPython.display import display, HTML
from tqdm import tqdm
import face_recognition
import dlib



filename="ITMD 525_03.MOV"   #filename
cap = cv2.VideoCapture(filename) #Read Frame
fps=cap.get(cv2.CAP_PROP_FPS) #Extract the frame per second (fps) 
print('Frames per second : ', fps,'FPS')
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)#get overall no# of frames in the video file
print('Frame count : ', frame_count)
sessions=pd.read_excel('session_info.xlsx', sheet_name='Sheet2')
sessions=sessions.rename(columns=lambda x:x.strip())
ses_crop=sessions.iloc[:, 0:7]#just getting FileName, StartTime, EndTime and SessionName
ses_crop



Frames per second :  29.97002997002997 FPS
Frame count :  93268.0


,File Name,Start time,End time,session_name,# of students in video,which assignments,# of students with wrong answers
0,ITMD 525_03.MOV,6.25,9.00,Information Overload & Solutions for Informati...,13,NaN,NaN
1,ITMD 525_03.MOV,9.22,10.09,Chunking,13,NaN,NaN
2,ITMD 525_03.MOV,10.10,10.29,Omission,13,NaN,NaN
3,ITMD 525_03.MOV,10.30,10.51,Queuing,13,NaN,NaN
4,ITMD 525_03.MOV,10.52,11.17,Capitulation,16,NaN,NaN
5,ITMD 525_03.MOV,11.40,12.19,Filtering & Giving examples about who uses RecSys,16,NaN,NaN
6,ITMD 525_03.MOV,16.20,19.45,RecSys,17,NaN,NaN
7,ITMD 525_03.MOV,19.46,22.14,How RecSys works (Netflix Example),18,NaN,NaN
8,ITMD 525_03.MOV,22.15,24.20,How it works,18,NaN,NaN
9,ITMD 525_03.MOV,24.22,28.07,Traditional Recommendation Algs,18,NaN,NaN


In [14]:
#selecting only sessions for which we have homework values 
ses_crop=ses_crop[np.isnan(ses_crop['# of students with wrong answers'])==False]
ses_crop.reset_index(drop=True, inplace=True)



In [15]:
ses_crop.shape
ses_crop

,File Name,Start time,End time,session_name,# of students in video,which assignments,# of students with wrong answers
0,ITMD 525_03.MOV,36.50,40.29,Rating Prediction,18,hw6,8.0
1,ITMD 525_03.MOV,40.29,46.13,Top-N Recommendation,18,hw 6,3.0
2,ITMD 525_03.MOV,47.26,51.52,Relevance Metrics,18,"hw2, hw6",3.0


In [16]:
#let's convert the start time and end time columns into proper time format i.e "%H:%M:%S"
from time import time
import math, time

starttime=[]
for i in range(len(ses_crop)):
    starttime.append(math.modf(ses_crop['Start time'][i])) # (0.5678000000000338, 1234.0)
print('\nstart time in disected form:', starttime)
#for i in range(len(starttime)):
#    start
    
seconds_s=[round(x[0]*100) for x in starttime]
secondsfromminutes_s=[x[1]*60 for x in starttime]

total_seconds_s=[]
for i in range(len(seconds_s)):
    total_seconds_s.append(seconds_s[i]+secondsfromminutes_s[i])
Start_time=[]
for i in range(len(total_seconds_s)):
    Start_time.append(time.strftime("%H:%M:%S", time.gmtime(total_seconds_s[i])))
print('\n\nStart time in Time Format:',Start_time)

#doing the same for End time
endtime=[]
for i in range(len(ses_crop)):
    endtime.append(math.modf(ses_crop['End time'][i])) # (0.5678000000000338, 1234.0)
print('\nEnd time in disected form:',endtime)

seconds_e=[round(x[0]*100) for x in endtime]
secondsfromminutes_e=[x[1]*60 for x in endtime]

total_seconds_e=[]
for i in range(len(seconds_e)):
    total_seconds_e.append(seconds_e[i]+secondsfromminutes_e[i])
End_time=[]
for i in range(len(total_seconds_e)):
    End_time.append(time.strftime("%H:%M:%S", time.gmtime(total_seconds_e[i])))
print('\n\nEnd time in Time Format:',End_time)

ses_len=[]
for i in range(len(total_seconds_s)):
    ses_len.append(total_seconds_e[i]-total_seconds_s[i])
    


start time in disected form: [(0.5, 36.0), (0.28999999999999915, 40.0), (0.259999999999998, 47.0)]


Start time in Time Format: ['00:36:50', '00:40:29', '00:47:26']

End time in disected form: [(0.28999999999999915, 40.0), (0.13000000000000256, 46.0), (0.5200000000000031, 51.0)]


End time in Time Format: ['00:40:29', '00:46:13', '00:51:52']


In [17]:
#assigning the time values in correct format to cropped session dataframe

ses_crop['Start time']=Start_time
ses_crop['End time']=End_time
ses_crop

C:\Users\munaw\AppData\Local\Temp\ipykernel_6768\3526766700.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ses_crop['Start time']=Start_time
C:\Users\munaw\AppData\Local\Temp\ipykernel_6768\3526766700.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ses_crop['End time']=End_time


,File Name,Start time,End time,session_name,# of students in video,which assignments,# of students with wrong answers
0,ITMD 525_03.MOV,00:36:50,00:40:29,Rating Prediction,18,hw6,8.0
1,ITMD 525_03.MOV,00:40:29,00:46:13,Top-N Recommendation,18,hw 6,3.0
2,ITMD 525_03.MOV,00:47:26,00:51:52,Relevance Metrics,18,"hw2, hw6",3.0


In [18]:
#slicing session-wise attendance for comparison at later stage
num_students=ses_crop.iloc[:,3:5]
num_students

,session_name,# of students in video
0,Rating Prediction,18
1,Top-N Recommendation,18
2,Relevance Metrics,18


In [19]:
#for each session we have to divide the video into 3 slices begin, dom, end
#for each of sub-sessions, we have to determine start frame and end frame for slicing the sessions
#let's determine startframe and endframe for begin session
startframe_begin=[]
for i in range(len(total_seconds_s)):
    startframe_begin.append(fps*(total_seconds_s[i]))

print(startframe_begin)

#after 5 seconds, begin subsession ends, therefore end frame of the begin subsession is located at (total_seconds_ofstarttime+5 seconds)*fps
endframe_begin=[]
for i in range(len(total_seconds_s)):
    endframe_begin.append(fps*(total_seconds_s[i]+5))

print('\n',endframe_begin)

#endframe_begin = startframe_dom
#endframe_dom = startframe_end
#startframe_end & endframe_end can be calculated with by using total_seconds_e
startframe_end=[]
for i in range(len(total_seconds_e)):
    startframe_end.append(fps*(total_seconds_e[i]-5))
    
endframe_end=[]
for i in range(len(total_seconds_e)):
    endframe_end.append(fps*total_seconds_e[i])

my_dict={'session_name':ses_crop['session_name'],'startframe_begin':startframe_begin, 'endframe_begin':endframe_begin,'startframe_end':startframe_end,'endframe_end':endframe_end, 'session_length':ses_len}
ses_timeframes=pd.DataFrame(my_dict 
             )

[66233.76623376623, 72797.2027972028, 85294.70529470529]

 [66383.61638361638, 72947.05294705294, 85444.55544455544]


In [20]:
ses_timeframes.reset_index(drop=True, inplace=True)
ses_timeframes

,session_name,startframe_begin,endframe_begin,startframe_end,endframe_end,session_length
0,Rating Prediction,66233.766234,66383.616384,72647.352647,72797.202797,219.0
1,Top-N Recommendation,72797.202797,72947.052947,82957.042957,83106.893107,344.0
2,Relevance Metrics,85294.705295,85444.555445,93116.883117,93266.733267,266.0


In [21]:
height=int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) #height
width=int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) #width


In [22]:
#let's now break down the video files into sessions as given in excel file
#further each session is going to be further divided into sub-sessions: begin, dom, end


tic=time.time()

#initializing lists for storing session-wise values for each subsession
begin_list = []
dom_list = []
end_list =[]
useless=[]
c =0

for i in tqdm(range(int(frame_count))):
    #following if-elif statements are only meants for determining value of c for iterating through the ses_timeframes table shown above
    
    if 66233.766234  <= i <= 72797.202797:
        c=0
    elif 72797.202797  <= i <= 83106.893107:
        c=1
    elif 85294.705295  <= i <= 93266.733267:
        c=2
  
       
    #read the frame from given video file
    ret, frame = cap.read()
    
    frame=cv2.resize(frame, (width,height))  #resize the frame
    
    if i>=ses_timeframes['startframe_begin'][c] and i<=ses_timeframes['endframe_begin'][c]:  
        begin_list.append([ses_timeframes['session_name'][c],cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)])            #output
    elif i>=ses_timeframes['endframe_begin'][c] and i<=ses_timeframes['startframe_end'][c]:
        dom_list.append([ses_timeframes['session_name'][c],cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)])
    elif i>=ses_timeframes['startframe_end'][c] and i<=ses_timeframes['endframe_end'][c]:  
        end_list.append([ses_timeframes['session_name'][c],cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)])
   # else:
    #    useless.append(frame)
        
#creating data frames out of lists
df_begin = pd.DataFrame(begin_list, columns=["session_name", "frame"])
df_dom = pd.DataFrame(dom_list, columns=["session_name", "frame"])
df_end = pd.DataFrame(end_list, columns=["session_name", "frame"]) 

#printing heads for begin, dom and end dfs
print('Begin Sessions Number of Frames', len(df_begin),  df_begin.head())
print('\n\nDOM Sessions Number of Frames', len(df_dom), df_dom.head())
print('\n\nEnd Sessions Number of Frames', len(df_end), df_end.head())  

tac=time.time()
print('Execution Time: ',round(tac-tic),' sec')


100%|███████████████████████████████████████████████████████████████████████████| 93268/93268 [02:03<00:00, 756.99it/s]


Begin Sessions Number of Frames 450         session_name                                              frame
0  Rating Prediction  [[[147, 133, 72], [147, 133, 72], [147, 133, 7...
1  Rating Prediction  [[[147, 133, 72], [147, 133, 72], [147, 133, 7...
2  Rating Prediction  [[[148, 134, 73], [148, 134, 73], [148, 134, 7...
3  Rating Prediction  [[[148, 134, 71], [148, 134, 71], [148, 134, 7...
4  Rating Prediction  [[[148, 134, 73], [148, 134, 73], [148, 134, 7...


DOM Sessions Number of Frames 23946         session_name                                              frame
0  Rating Prediction  [[[151, 139, 77], [151, 139, 77], [151, 139, 7...
1  Rating Prediction  [[[150, 138, 76], [150, 138, 76], [150, 138, 7...
2  Rating Prediction  [[[150, 138, 76], [150, 138, 76], [150, 138, 7...
3  Rating Prediction  [[[150, 138, 76], [150, 138, 76], [150, 138, 7...
4  Rating Prediction  [[[150, 138, 74], [150, 138, 74], [150, 138, 7...


End Sessions Number of Frames 449         session_name      

In [23]:
#just checking the number of frames per subsession 
begin_sessions=df_begin.groupby(['session_name'], sort=False).count()
display(HTML(begin_sessions.to_html()))
dom_sessions=df_dom.groupby(['session_name'],sort=False).count()
display(HTML(dom_sessions.to_html()))
end_sessions=df_end.groupby(['session_name'],sort=False).count()
display(HTML(end_sessions.to_html()))

,frame
session_name,
Rating Prediction,150
Top-N Recommendation,150
Relevance Metrics,150


,frame
session_name,
Rating Prediction,6264
Top-N Recommendation,10010
Relevance Metrics,7672


,frame
session_name,
Rating Prediction,150
Top-N Recommendation,149
Relevance Metrics,150


In [24]:
### HISTOGRAM OF GRADIENT ALGORITHM ###
#creating function for applying HOG algorithm on each frame to detect faces

def NumFaces_HOG (image):
    
    face_locations = face_recognition.face_locations(image,  number_of_times_to_upsample=3, model='hog')

    num_faces = len(face_locations)
    return num_faces
    


In [25]:
#let's apply HOG for face detection

#detect number of faces for each frame of begin subsessions
face_list_begin=[]
for i in tqdm(range(len(df_begin))):
    frame_1=df_begin['frame'][i]
    num_faces=NumFaces_HOG(frame_1)
    face_list_begin.append([df_begin['session_name'][i], num_faces])
          
faces_begin_hog = pd.DataFrame(face_list_begin, columns=["session_name", "no_of_Faces"])

#detect number of faces for each frame of dom subsessions
face_list_dom=[]
for i in tqdm(range(len(df_dom))):
    frame_1=df_dom['frame'][i]
    num_faces=NumFaces_HOG(frame_1)
    face_list_dom.append([df_dom['session_name'][i], num_faces])
          
faces_dom_hog = pd.DataFrame(face_list_dom, columns=["session_name", "no_of_Faces"])

#detect number of faces for each frame of end subsessions
face_list_end=[]
for i in tqdm(range(len(df_end))):
    frame_1=df_end['frame'][i]
    num_faces=NumFaces_HOG(frame_1)
    face_list_end.append([df_end['session_name'][i], num_faces])
          
faces_end_hog = pd.DataFrame(face_list_end, columns=["session_name", "no_of_Faces"])

#compute MINIMUM of number detected faces in each subsession
face_begin_min_hog=faces_begin_hog.groupby(['session_name'],sort=False).min()
face_dom_min_hog=faces_dom_hog.groupby(['session_name'],sort=False).min()
face_end_min_hog=faces_end_hog.groupby(['session_name'],sort=False).min()
#compute MAXIMUM of number detected faces in each subsession
face_begin_max_hog=faces_begin_hog.groupby(['session_name'],sort=False).max()
face_dom_max_hog=faces_dom_hog.groupby(['session_name'],sort=False).max()
face_end_max_hog=faces_end_hog.groupby(['session_name'],sort=False).max()
#compute AVERAGE of number detected faces in each subsession
face_begin_avg_hog=faces_begin_hog.groupby(['session_name'],sort=False).mean()
face_dom_avg_hog=faces_dom_hog.groupby(['session_name'],sort=False).mean()
face_end_avg_hog=faces_end_hog.groupby(['session_name'],sort=False).mean()

100%|████████████████████████████████████████████████████████████████████████████████| 449/449 [16:30<00:00,  2.21s/it]


In [33]:


#let's analyze variation in [minimum, average, and maximum] number of faces detected in the begin session 
merged_2_begin_hog=face_begin_min_hog.merge(face_begin_avg_hog, on='session_name', how='outer')
merged_all_begin_hog=merged_2_begin_hog.merge(face_begin_max_hog, on='session_name',how='outer')
num_students_begin_hog=merged_all_begin_hog.merge(num_students, on='session_name',how='outer')
num_students_begin_hog.columns = ['begin session name', 'minimum', 'average', 'maximum', 'actual']
display(HTML(num_students_begin_hog.to_html()))

#let's analyze variation in [minimum, average, and maximum] number of faces detected in the dominant session 
merged_2_dom_hog=face_dom_min_hog.merge(face_dom_avg_hog, on='session_name', how='outer')
merged_all_dom_hog=merged_2_dom_hog.merge(face_dom_max_hog, on='session_name',how='outer')
num_students_dom_hog=merged_all_dom_hog.merge(num_students, on='session_name',how='outer')
num_students_dom_hog.columns = ['begin session name', 'minimum', 'average', 'maximum', 'actual']
display(HTML(num_students_dom_hog.to_html()))

#let's analyze variation in [minimum, average, and maximum] number of faces detected in the dominant session 
merged_2_end_hog=face_end_min_hog.merge(face_end_avg_hog, on='session_name', how='outer')
merged_all_end_hog=merged_2_end_hog.merge(face_end_max_hog, on='session_name',how='outer')
num_students_end_hog=merged_all_end_hog.merge(num_students, on='session_name',how='outer')
num_students_begin_hog.columns = ['begin session name', 'minimum', 'average', 'maximum', 'actual']
num_students_end_hog.columns = ['begin session name', 'minimum', 'average', 'maximum', 'actual']
display(HTML(num_students_end_hog.to_html()))

,begin session name,minimum,average,maximum,actual
0,Rating Prediction,5,6.840000,10,18
1,Top-N Recommendation,4,6.246667,9,18
2,Relevance Metrics,3,6.500000,11,18


,begin session name,minimum,average,maximum,actual
0,Rating Prediction,2,6.361909,11,18
1,Top-N Recommendation,1,6.614985,12,18
2,Relevance Metrics,1,6.161496,12,18


,begin session name,minimum,average,maximum,actual
0,Rating Prediction,4,5.620000,9,18
1,Top-N Recommendation,4,6.161074,8,18
2,Relevance Metrics,3,3.920000,6,18
